In [71]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score

In [67]:
df = pd.read_csv('D:/Kaggle/train.csv')
df1 = pd.read_csv(('D:/Kaggle/test.csv'))
df.comment_text = df.comment_text.fillna('')
df1.comment_text = df1.comment_text.fillna('')

In [68]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [69]:
cv = TfidfVectorizer(ngram_range=(2,2),tokenizer=tokenize,min_df=3,max_df=0.9,strip_accents='unicode',use_idf=1,smooth_idf=1,sublinear_tf=1)
train_data = df.comment_text
cv.fit(train_data)
train_data = cv.transform(train_data)
test_data = cv.transform(df1.comment_text)

In [72]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

submission = pd.DataFrame.from_dict({'id': df1['id']})

for class_name in  class_names:
    train_label = df[class_name].astype('int')
    
    mnb = MultinomialNB()
    cv_score = np.mean(cross_val_score(mnb, train_data, train_label, cv=3,scoring='roc_auc'))
    
    mnb.fit(train_data,train_label)
    print(classname,cv_score)
    submission[class_name] = mnb.predict_proba(test_data)[:,1]
    

submission.to_csv('D:\Kaggle\submission.csv',index=False)